<a href="https://colab.research.google.com/github/mathluva/BERT-QA/blob/main/BERT_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install torch

     |████████████████████████████████| 2.2MB 19.5MB/s 
     |████████████████████████████████| 870kB 48.0MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=b9d1274b012c36ae7b3fbd0ac2b0f0d8909c4dc5acaa699aacdecb6bedbd78b1
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [3]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np

In [4]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [5]:
#BERT tokenizer
tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [13]:
def bert_answering_machine(question, passage, max_len=512):

    #tokenize input question and passage, add special tokens
    input_ids = tokenizer_for_bert.encode(question, passage, max_length=max_len, truncation = True)


    cls_index = input_ids.index(102) #index of first SEP token
    len_question = cls_index +1 #length of question (sentence 1)
    len_answer = len(input_ids) - len_question #length of sentence 2

    #segment ids to distinguish between the two sentences
    #apply 0 for sentence 1 and apply 1 for sentence 2
    segment_ids = [0]*len_question + [1]*len_answer

    


    #start and end scores for answer, convert input to torch tensors
    start_token_scores, end_token_scores = model(torch.tensor([input_ids]), token_type_ids =torch.tensor([segment_ids]))

    #convert tensors to numpy arrays for calculations
    #start_token_scores = start_token_scores.detach().numpy().flatten()
    #end_token_scores = end_token_scores.detach().numpy().flatten()

    #pick start/end index of answer based on highest score
    answer_start_index = torch.argmax(start_token_scores)
    answer_end_index = torch.argmax(end_token_scores)

    #get scores for start/end token of answer and round to 2 decimal places
    #start_token_score = np.round(start_token_scores[answer_start_index], 2)
    #end_token_score = np.round(start_token_scores[answer_end_index], 2)

    #covert token ids to tokens
    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)
    
    #combine words split using ##
    answer = tokens[answer_start_index]
    for i in range(answer_start_index+1, answer_end_index+1):
        if tokens[i][0:2]=="##":
            answer +=tokens[i][2:]
        else:
            answer += " "+token[i]
    
    #patterns indicating that BERT does not have answer from the passage for question asked
    if (answer_start_index==0) or (start_token_score<0) or (answer=='[SEP]') or (answer_end_index < answer_start_index):
        answer= "Sorry, BERT could not find answer passage."
    
    return (answer_start_index, answer_end_index, start_token_score, end_token_score, answer)


In [25]:
import tensorflow as tf

In [26]:
tiffany_mom = tf.convert_to_tensor("Tiffany was born in Norfolk, VA.  Her mom was born in Columbus, Ohio.  She has lived in Fl, GA and NJ.  She currently resides in Raleigh,NC.")

In [27]:
question = tf.convert_to_tensor("Where does Tiffany's mom live?")


In [29]:
bert_answering_machine("where does Tiffany's mom live?", tiffany_mom, 50)

ValueError: ignored